In [2]:
# run this same experiment again but instead of 2,0,0 -> 0,0,2 have 2,0,0 -> 0,1,1 this would be cool 
# declaring all dependancies    
import qutip as qt
import numpy as np
from quantum_logical.pulsesim import QuantumSystem, Pulse
from quantum_logical.pulsesim.mode import QubitMode, SNAILMode, CavityMode
from quantum_logical.pulsesim.build_hamiltonian import Build_hamiltonian
import matplotlib.pyplot as plt
from quantum_logical.channel import AmplitudeDamping, PhaseDamping
from quantum_logical.trotter import TrotterGroup
from quantum_logical.interaction import ConversionGainInteraction
from qutip import Options
from tqdm import tqdm
from progressbar import progressbar
from threading import Thread
from multiprocessing.pool import Pool, ThreadPool
from numpy.linalg import eig
from quantum_logical.creating_operators import Creating_operators
from quantum_logical.operators import selective_destroy

# opts = Options(nsteps=1e6, atol=1e-8, rtol=1e-6)
# p_bar = qt.ui.TextProgressBar()  # None
opts = {"nsteps": 1e6, "atol": 1e-8, "rtol": 1e-6, "progress_bar": "text"}

In [3]:
fund_ops_list = []
H = Creating_operators(qubit_count=2, qubit_dim=3)
fund_ops_list.append(H.create_ops_unconventional_annhilation(from_level=0, to_level=1))  # this is not necessary with the new addition for the operator 
fund_ops_list.append(H.create_ops_unconventional_annhilation(from_level=1, to_level=2))

# # list of operators based on the fundamental single qubit operators 
# ops_list = []
# ops_list.append(qt.tensor(fund_ops_list[0], fund_ops_list[1].dag()))
# ops_list.append(qt.tensor(fund_ops_list[1], fund_ops_list[0].dag()))
# ops_list.append(qt.tensor(fund_ops_list[1].dag(), fund_ops_list[0]))
# ops_list.append(qt.tensor(fund_ops_list[0].dag(), fund_ops_list[1]))

# list of operators based on the full dimension operators
ops_list = []
ops_list.append(H.creating_ops_full_dim(qubit1_from=1, qubit1_to=0, qubit2_from=1, qubit2_to=2))
ops_list.append(H.creating_ops_full_dim(qubit1_from=2, qubit1_to=1, qubit2_from=0, qubit2_to=1))
ops_list.append(H.creating_ops_full_dim(qubit1_from=1, qubit1_to=2, qubit2_from=1, qubit2_to=0))
ops_list.append(H.creating_ops_full_dim(qubit1_from=0, qubit1_to=1, qubit2_from=2, qubit2_to=1))

ops_list1 = []
ops_list1.append(H.create_ops_unconventional_annhilation(from_level=1, to_level=0))
ops_list1.append(H.create_ops_unconventional_annhilation(from_level=2, to_level=1))

# this doesnt work yet but it is necessary for this to work
# ops_list2 = []
# ops_list1 = np.zeros((9,9), dtype=complex)
# for i in range(len(ops_list)):
#     for j in range(ops_list[i].shape[0]):
#         for k in range(ops_list[i].shape[0]):
#             ops_list1[j,k] = ops_list[i][j,k]
#     ops_list2.append(ops_list1)

# for i in range(len(ops_list2)):
#     ops_list2[i] = qt.Qobj(ops_list2[i])

# ops_list[0].shape[0]


In [4]:
selective_destroy(levels=3, from_level=1, to_level=0) == ops_list1[0]

True

In [5]:
# put in the target cnot 
# check that these operations give a cnot
H_new = ConversionGainInteraction(gg=np.pi / 4, gc= np.pi / 4, transmon_levels=3)
U_cnot_no_time = H_new.construct_U(1.0)
# H_mine = (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0].dag()) + (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0]) + (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0].dag()).dag() + (np.pi / 4) * qt.tensor(ops_list1[0], ops_list1[0]).dag()
# U_mine = H.propagator(H=H_mine, T=1)
# U_mine

In [6]:
U_cnot_no_time

Quantum object: dims=[[3, 3], [3, 3]], shape=(9, 9), type='oper', dtype=Dense, isherm=False
Qobj data =
[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        -0.70710678j 0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.70710678+0.j         0.        +0.j
  0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        +0.j         1.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.        +0.j
  0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.        +0.j         0.70710678+0.j  

In [7]:
# full hamiltonian with snail terms
# H_no_time = 

In [11]:
# make the table of coefficients and then create the pulse object (smooth box is probably the best idea)
gg_coeff = gc_coeff = np.linspace(0, 2 * np.pi, 10)
time_coeff = np.linspace(10,100,10)
coeffs = [(i, j, k) for i in gg_coeff for j in gc_coeff for k in time_coeff]
# create the gain and conversion operators 

# def sim_task_gate(coeffs):
#      gg, gc, time = coeffs

#      tlist = np.linspace(0, time, 20)
#      # build the pulse
#      args = {"shape": Pulse.box, "shape_params": {"t0": time * .05, "width": time * .9}}
#      pulse1 = Pulse(amp = gg)
#      pulse2 = Pulse(amp = gc)

#      H = [[qt.tensor(ops_list1[0], ops_list1[0].dag()), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]), pulse2.drive],
#           [qt.tensor(ops_list1[0], ops_list1[0].dag()).dag(), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]).dag(), pulse2.drive]]

#      U_prop_target = qt.propagator(H=H, t=tlist, args=args)

#      fid = qt.process_fidelity(U_prop_target, U_cnot_no_time)

#      return fid 

# results = qt.parallel.parallel_map(task=sim_task_gate, values=coeffs, progress_bar=True)
fid = []

for i in tqdm(coeffs):
     tlist = np.linspace(0, i[2], 20)
     # build the pulse
     args = {"shape": Pulse.box, "shape_params": {"t0": i[2] * .05, "width": i[2] * .9}}
     pulse1 = Pulse(omega=0, amp = i[1])
     pulse2 = Pulse(omega=0, amp = i[0])

     H = [[qt.tensor(ops_list1[0], ops_list1[0].dag()), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]), pulse2.drive],
          [qt.tensor(ops_list1[0], ops_list1[0].dag()).dag(), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]).dag(), pulse2.drive]]

     U_prop_target = qt.propagator(H=H, t=tlist, args=args)

     fid.append(qt.average_gate_fidelity(U_prop_target, U_cnot_no_time))


# how can fidelity be best measured between the propagators and make sure that you are not getting inappropriate values. 
# what is the frequency of the pulse?


SyntaxError: invalid syntax (253029997.py, line 35)

In [ ]:
# use the abbreviated hamiltonian 
H_abv = [qt.tensor(ops_list1[0], ops_list1[0].dag()), qt.tensor(ops_list1[0], ops_list1[0]),
        qt.tensor(ops_list1[0], ops_list1[0].dag()).dag(), qt.tensor(ops_list1[0], ops_list1[0]).dag(),
        qt.tensor(ops_list1[1], ops_list1[1].dag()), qt.tensor(ops_list1[1], ops_list1[1]),
        qt.tensor(ops_list1[1], ops_list1[1].dag()).dag(), qt.tensor(ops_list1[1], ops_list1[1]).dag()]
# this hamiltonian contains the conversion and gain terms that go from 1 to 0 and from 1 to 2
# do the analysis as before and find how high of a fidelity you can get to get the results that give a matrix that makes sense

In [12]:
fid1 = []
# time is set 
freq1 = freq2 = np.linspace(0, 4, 10)
amp1 = amp2 = np.linspace(0, 4, 10)
coeffs2 = [(i, j, k, l) for i in freq1 for j in freq2 for k in amp1 for l in amp2]
full_time = 100
tlist = np.linspace(0, full_time, 20)
for i in tqdm(coeffs2):
    #  tlist = np.linspace(0, i[2], 20)
     # build the pulse
     args = {"shape": Pulse.box, "shape_params": {"t0": full_time * .05, "width": full_time * .9}}
     pulse1 = Pulse(omega=i[0], amp = i[2])
     pulse2 = Pulse(omega=i[1], amp = i[3])

     H = [[qt.tensor(ops_list1[0], ops_list1[0].dag()), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]), pulse2.drive],
          [qt.tensor(ops_list1[0], ops_list1[0].dag()).dag(), pulse1.drive], [qt.tensor(ops_list1[0], ops_list1[0]).dag(), pulse2.drive]]

     U_prop_target = qt.propagator(H=H, t=tlist, args=args)

     fid1.append(qt.average_gate_fidelity(U_prop_target, U_cnot_no_time))

100%|██████████| 10000/10000 [07:18<00:00, 22.80it/s]


In [14]:
max(fid1)

16.371501582626244

In [9]:
# use full hamiltonian and do not optimize over the time but instead do what you have done in the past and optimize over the frequency and the amplitude of the pulse and see if you get something that is plausible 


In [ ]:
# maybe the best way to find this fidelity is to see how it moves in the weyl chamber and compare that to what you need to find 
# ask evan if he has code for how things behave in the weyl chamber 